In [1]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geopyspark.geotrellis.catalog import _construct_catalog, _mapped_cached
from geopyspark.geotrellis.constants import SPATIAL, TILE
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
from geonotebook.wrappers.vector import GeoJsonData
import numpy as np
from shapely.geometry import Polygon, mapping
from shapely.ops import transform
from functools import partial
import pyproj
from PIL import Image

In [5]:
def rgb_render(tile):
    alpha = np.full((256, 256), 0xFF)
    alpha[np.ma.where((tile[0] == 0) & (tile[1] == 0) & (tile[2] == 0))] =0x00
    arr = np.dstack([tile[0], tile[1], tile[2], alpha]).astype('uint8')
    return Image.fromarray(arr, mode='RGBA')

def remove_layer(name):
    to_remove = []
    for l in M.layers:
        if l.name == name:
            to_remove.append(l)
    for x in to_remove:
        M.remove_layer(x)
        
def safe_divide(a, b):
    """
    Avoid divide by zero
    http://stackoverflow.com/questions/26248654/numpy-return-0-with-divide-by-zero
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(a, b)
        c[c == np.inf] = 0
        c = np.nan_to_num(c)
        return c


In [2]:
sc = SparkContext(appName="Potsdam Viz")
geopysc = GeoPyContext(sc)

In [3]:
catalog_uri = "s3://otid-data/viz/catalog"
label_layer_name = "isprs-potsdam-labels"
imagery_layer_name = "isprs-potsdam-imagery-rgb"
rir_layer_name = "isprs-potsdam-imagery-rir"

In [4]:
M.set_center(13.04951140975037, 52.40398631048631, 15)
M.set_zoom_range(0, 22)

In [6]:
def add_imagery_layer():
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      imagery_layer_name,
                                      SPATIAL)
    M.add_layer(data, name="aerial", render_tile=rgb_render, max_zoom=22)

In [7]:
def add_ndvi_analysis(threshold):
    """
    Layer which indicates what pixels have an NDVI threshold above
    a specific value but are not labeled as low vegetation or tree,
    also pixels that are below the threshold that are labeled
    low vegitation or tree.
    """
    colors = { 1: "#FF0000FF", # High NDVI not a tree or low veg [RED]
               2: "#0000FFFF", # Low NDVI and a tree [BLUE]
               3: "#00FF00FF", # High NDVI and a tree [GREEN]
               4: "#91E6F2FF", # Low NDVI and low veg [LIGHT BLUE]
               5: "#77C168FF"  # High NDVI and veg [LIGHT GREEN]
             } 
    cmap_render = single_band_render_from_color_map(colors)
    def render_tile(tiles):
        rir, labels = tiles[0], tiles[1]
        red = rir[0].astype('float64')
        ir = rir[1].astype('float64')
        ndvi = safe_divide((ir - red), (ir + red))
        arr = np.zeros((256,256))
  
        high_ndvi_no_veg = np.ma.where((ndvi >= threshold) & \
                                   ((labels[0] == 0xFF) | 
                                    (labels[1] == 0x00)))
        arr[high_ndvi_no_veg] = 1
        
        low_ndvi_tree = np.ma.where((ndvi < threshold) & \
                                       (labels[0] == 0x00) & \
                                       (labels[1] == 0xFF) & \
                                       (labels[2] == 0x00))
        arr[low_ndvi_tree] = 2
        
        high_ndvi_tree = np.ma.where((ndvi >= threshold) & \
                                    (labels[0] == 0x00) & \
                                    (labels[1] == 0xFF) & \
                                    (labels[2] == 0x00))
        arr[high_ndvi_tree] = 3  
        
        low_ndvi_low_veg = np.ma.where((ndvi < threshold) & \
                                       (labels[0] == 0) & \
                                       (labels[1] == 0xFF) & \
                                       (labels[2] == 0xFF) & \
                                       (arr == 0))

        arr[low_ndvi_low_veg] = 4

        high_ndvi_low_veg = np.ma.where((ndvi >= threshold) & \
                                        (labels[0] == 0x00) & \
                                        (labels[1] == 0xFF) & \
                                        (labels[2] == 0xFF) & \
                                        (arr == 0))
        arr[high_ndvi_low_veg] = 5
        
        return cmap_render(arr)
    
    layers = [rir_layer_name, label_layer_name]
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      layers,
                                      SPATIAL)
    remove_layer("ndvi-analysis")
    M.add_layer(data, name="ndvi-analysis", render_tile=render_tile, max_zoom=22)

In [8]:
add_imagery_layer()

::1 - - [2017-05-29 00:04:35] "GET /tile/15/17571/10763.png HTTP/1.1" 200 111284 1.471694
::1 - - [2017-05-29 00:04:35] "GET /tile/15/17571/10765.png HTTP/1.1" 200 18284 0.183385
::1 - - [2017-05-29 00:04:36] "GET /tile/15/17570/10764.png HTTP/1.1" 200 92099 0.233079
::1 - - [2017-05-29 00:04:36] "GET /tile/15/17572/10762.png HTTP/1.1" 200 54933 0.195412
::1 - - [2017-05-29 00:04:36] "GET /tile/15/17571/10762.png HTTP/1.1" 200 35348 0.194634
::1 - - [2017-05-29 00:04:36] "GET /tile/15/17572/10765.png HTTP/1.1" 200 15348 0.172290
::1 - - [2017-05-29 00:04:36] "GET /tile/15/17573/10763.png HTTP/1.1" 200 97404 0.276912
::1 - - [2017-05-29 00:04:37] "GET /tile/15/17573/10764.png HTTP/1.1" 200 87693 0.212163
::1 - - [2017-05-29 00:04:37] "GET /tile/15/17573/10762.png HTTP/1.1" 200 19964 0.175482
::1 - - [2017-05-29 00:04:37] "GET /tile/15/17570/10762.png HTTP/1.1" 500 2971 0.076593
::1 - - [2017-05-29 00:04:37] "GET /tile/15/17570/10765.png HTTP/1.1" 200 20500 0.148557
::1 - - [2017-05-29 0

::1 - - [2017-05-29 00:12:14] "GET /tile/19/281147/172220.png HTTP/1.1" 200 132512 0.195969
::1 - - [2017-05-29 00:12:14] "GET /tile/19/281147/172223.png HTTP/1.1" 200 132320 0.268915
::1 - - [2017-05-29 00:12:14] "GET /tile/19/281147/172219.png HTTP/1.1" 200 127568 0.283882
::1 - - [2017-05-29 00:12:14] "GET /tile/19/281148/172221.png HTTP/1.1" 200 130979 0.218470


In [10]:
add_ndvi_analysis(0.1)

::1 - - [2017-05-29 00:12:15] "GET /shutdown HTTP/1.1" 500 412 0.002902
::1 - - [2017-05-29 00:12:19] "GET /tile/19/281153/172221.png HTTP/1.1" 200 5713 0.506459
::1 - - [2017-05-29 00:12:19] "GET /tile/19/281154/172221.png HTTP/1.1" 200 7077 0.422379
::1 - - [2017-05-29 00:12:20] "GET /tile/19/281153/172220.png HTTP/1.1" 200 6148 0.347648
::1 - - [2017-05-29 00:12:20] "GET /tile/19/281153/172222.png HTTP/1.1" 200 5511 0.319469
::1 - - [2017-05-29 00:12:20] "GET /tile/19/281154/172220.png HTTP/1.1" 200 5283 0.316983
::1 - - [2017-05-29 00:12:21] "GET /tile/19/281154/172222.png HTTP/1.1" 200 6185 0.360510
::1 - - [2017-05-29 00:12:21] "GET /tile/19/281155/172221.png HTTP/1.1" 200 3099 0.331870
::1 - - [2017-05-29 00:12:21] "GET /tile/19/281152/172221.png HTTP/1.1" 200 7313 0.383466
::1 - - [2017-05-29 00:12:22] "GET /tile/19/281152/172222.png HTTP/1.1" 200 6734 0.354656
::1 - - [2017-05-29 00:12:22] "GET /tile/19/281155/172222.png HTTP/1.1" 200 7011 0.279721
::1 - - [2017-05-29 00:12:22

In [ ]:
remove_layer("ndvi-analysis")

In [ ]:
remove_layer("aerial")